In [52]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import matplotlib
import matplotlib.pyplot as plt
import napari
import cupy as cp
from cupyx.scipy.ndimage import gaussian_filter, histogram
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


### Managing memory for GPU use

In [ ]:
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()

# Create an array on CPU.
# NumPy allocates 400 bytes in CPU (not managed by CuPy memory pool).
a_cpu = numpy.ndarray(100, dtype=numpy.float32)
print(a_cpu.nbytes)                      # 400

# You can access statistics of these memory pools.
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

# Transfer the array from CPU to GPU.
# This allocates 400 bytes from the device memory pool, and another 400
# bytes from the pinned memory pool.  The allocated pinned memory will be
# released just after the transfer is complete.  Note that the actual
# allocation size may be rounded to larger value than the requested size
# for performance.
a = cupy.array(a_cpu)
print(a.nbytes)                          # 400
print(mempool.used_bytes())              # 512
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

# When the array goes out of scope, the allocated device memory is released
# and kept in the pool for future reuse.
a = None  # (or `del a`)
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

# You can clear the memory pool by calling `free_all_blocks`.
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0

### Reading in images

In [2]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [3]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [10]:
viewer = napari.view_image(GMO_1ng_imgs[1], name= 'test', scale = [5.0,0.1,0.1])

## Plan: preprocess, threshold signal, connected components, post-process objects, measure volume and SA to get ratio, measure fluorescence values, PCA on volume/SA to find threshold for splitting into active and inactive state?

#### Trying to get GPU use working

In [ ]:
a_np = np.arange(10)
s = cp.cuda.Stream()
with s:
    a_cp = cp.asarray(a_np)  # H2D transfer on stream s
    b_cp = cp.sum(a_cp)      # kernel launched on stream s
    assert s == cp.cuda.get_current_stream()

In [49]:
test_img = GMO_1ng_imgs[1]
test_img = np.asarray(test_img)
test_img.shape

(42, 2048, 2048)

In [53]:
#push img to GPU as array

s = cp.cuda.Stream()
with s:
    test_array = cp.asarray(test_img)
    gaus_bg = gaussian_filter(test_array, sigma = (12,683,683))
    #bg_sub = test_array - gaus_bg
    #bg_sub = cp.asnumpy(bg_sub)
    assert s == cp.cuda.get_current_stream()

CompileException: nvrtc: error: failed to open nvrtc-builtins64_128.dll.
  Make sure that nvrtc-builtins64_128.dll is installed correctly.

In [ ]:
#testing gaussian background subtraction

In [ ]:
#test_img.shape
gaus_bg = sk.filters.gaussian(test_img,sigma=(12,683,683))

In [17]:
median_filter = sk.filters.median(test_img)

In [18]:
viewer = napari.view_image(test_img, name= 'test_img')
viewer.add_image(median_filter, name='median_filter')

<Image layer 'median_filter' at 0x1f473e93740>

In [19]:
otsu = sk.filters.threshold_otsu(median_filter)
iso = sk.filters.threshold_isodata(median_filter)
li = sk.filters.threshold_li(median_filter)
local = sk.filters.threshold_local(median_filter)
mean = sk.filters.threshold_mean(median_filter)
minimum = sk.filters.threshold_minimum(median_filter)
niblack = sk.filters.threshold_niblack(median_filter)
sauvola = sk.filters.threshold_sauvola(median_filter)
triangle = sk.filters.threshold_triangle(median_filter)
yen = sk.filters.threshold_yen(median_filter)

C:\ProgramData\anaconda3\envs\jupyterlab-env\Lib\site-packages\skimage\filters\thresholding.py:561: RuntimeWarning: divide by zero encountered in divide
  higher = (csum_intensity[-1] - csum_intensity[:-1]) / csumh[:-1]


In [27]:
otsu_mask = median_filter > otsu
iso_mask = median_filter > iso
li_mask = median_filter > li
local_mask = median_filter > local
mean_mask = median_filter > mean
min_mask = median_filter > minimum
niblack_mask = median_filter > niblack
sauvola_mask = median_filter > sauvola
triangle_mask = median_filter > triangle
yen_mask = median_filter > yen

In [43]:
viewer = napari.view_image(test_img, name= 'test_img')
viewer.add_image(median_filter, name='median_filter')
viewer.add_image(otsu_mask, name='otsu')
viewer.add_image(iso_mask, name='iso')
viewer.add_image(li_mask, name='li')
viewer.add_image(local_mask, name='local')
viewer.add_image(mean_mask, name='mean')
viewer.add_image(min_mask, name='min')
viewer.add_image(niblack_mask, name='niblack')
viewer.add_image(sauvola_mask, name='sauvola')
viewer.add_image(triangle_mask, name='triangle')
viewer.add_image(yen_mask, name='yen')

<Image layer 'yen' at 0x1f5e0d7b4a0>

### Need to add in additional preprocessing

In [ ]:
#Testing Rolling Ball background subtraction
rb_test = sk.restoration.rolling_ball(GMO_1ng_imgs[0],num_threads=12)

In [ ]:
viewer = napari.view_image(rb_test, name= "rb=100")
viewer.add_image(GMO_1ng_imgs[0], name='og')

In [29]:
import random

In [33]:
l= range(len(CMO_imgs))
i = random.randint(0,8)
print(l)
print(i)

range(0, 8)
5


In [44]:
def median_filter_and_threshold(i, img_list):
    median = sk.filters.median(img_list[i])
    mask = median > sk.filters.threshold_otsu(median)
    return mask

In [45]:
i = random.randint(0,8)
CMO_mask = median_filter_and_threshold(i, CMO_imgs)
GMO_1ng_mask = median_filter_and_threshold(i, GMO_1ng_imgs)
GMO_2ng_mask = median_filter_and_threshold(i, GMO_2ng_imgs)

In [46]:
viewer = napari.view_image(GMO_2ng_imgs[i], name= 'img')
viewer.add_image(GMO_2ng_li_mask, name='Mask')

<Image layer 'Mask' at 0x1f5e14f2930>